In [1]:
# Import neccesary Libraries
import os
import pandas as pd
import numpy as np
from getpass import getpass
import mysql.connector

## Extract State Paths

In [2]:
# here i want to extract the state paths so i can be easy to loop over and retrieve the json files

# Get the data path
directory = 'C:\\Users\\a\\OneDrive\\Desktop\\Github_projects\\pulse\\data\\'


# Function to extract every path subfolders that contains state
def state_path(directory):
    state_paths = []
    for root, dirs, files in os.walk(directory):
        if 'state' in dirs:
            state_dir = os.path.join(root, 'state')
            state_dir = state_dir.replace('\\', '/')
            state_paths.append(state_dir)
    return state_paths

# Call the function to extract the state paths
state_paths = state_path(directory)

In [3]:
# Let's check the paths
for path in state_paths:
    print(path)

C:/Users/a/OneDrive/Desktop/Github_projects/pulse/data/aggregated/transaction/country/india/state
C:/Users/a/OneDrive/Desktop/Github_projects/pulse/data/aggregated/user/country/india/state
C:/Users/a/OneDrive/Desktop/Github_projects/pulse/data/map/transaction/hover/country/india/state
C:/Users/a/OneDrive/Desktop/Github_projects/pulse/data/map/user/hover/country/india/state
C:/Users/a/OneDrive/Desktop/Github_projects/pulse/data/top/transaction/country/india/state
C:/Users/a/OneDrive/Desktop/Github_projects/pulse/data/top/user/country/india/state


Here we can see that we have 6 paths that contains state subfolders, but something common about it is that they're categorized into three, Agggregate, Map, and Top, they all contain two types of data User data and Transaction data

## Extract Data

Since we have all paths the next thing is to extract the data set, using the path. But the state subfolders do not contain the data, rather it contains each state in India subfolders, then each state also contains year subfolders which contains 4 sets of Json file which represents the each quater which is what we want to extract. The best thing to do is to create a function for each path which will iterate over all this subfolder then extrat the Json files orderly, and we will be able to keep track of Each State, Year, and Quater of the files we're extracting. The function is going to return a dataframe.

**Aggregate Transaction Data**

In [4]:
# Create the function to extract Aggregate Transaction data, this will accept the path to the aggregate transaction 

def extract_agg_transact_data(path):
    #Create a dictionary which contains an empty list to store each items that will be retrieved
    agg_data = {
        'State':[], 'Year':[], 'Quarter':[],
        'Payment_Type':[], 'Total_Transaction':[], 'Transaction_Value':[]
    }
    # loop over each state in the states subfolder
    for state in os.listdir(path):
        # Add each state to the path
        state_path = f"{path}/{state}"
        # loop over each year in the state path derived above
        for year in os.listdir(state_path):
            # add each year to the path 
            year_path = f"{state_path}/{year}"
            # loop over the json files
            for json_file in os.listdir(year_path):
                # add each file to the path
                json_path = f"{year_path}/{json_file}"
                # Retreive what quater we're currently extracting, by separating the .json handle
                json_f = json_file.split('.json')[0]
                # convert to a dataframe for easier extraction
                df = pd.read_json(json_path)
                # Chose exactly where we want to extract the data on the dataframe
                data_to_extract = df['data']['transactionData']
                # Loop through the data to extract what we want
                for data in data_to_extract:
                    # Extract the payment type and append it to the appropriate list in the dictionary
                    agg_data['Payment_Type'].append(data['name'])
                    # Extract the Total Transaction and append it to the appropriate list in the dictionary
                    agg_data['Total_Transaction'].append(data['paymentInstruments'][0]['count'])
                    # Extract the Total Value and append it to the appropriate list in the dictionary
                    agg_data['Transaction_Value'].append(data['paymentInstruments'][0]['amount'])
                    # Here we retrieve the current state we're extracting from remove uneccesary symbols, clean and Capitalize then append it to the appropriate list in the dictionary
                    agg_data['State'].append(state.title().replace('-', ' ').replace('&', 'and'))
                    # Here we retrieve the year and append it to the appropriate list in the dictionary
                    agg_data['Year'].append(year)
                    # Reterieve the Quater we're curently extracting from and append it to the appropriate list in the dictionary
                    agg_data['Quarter'].append(json_f)
    # We convert the Dictionary into a Dataframe
    df_tran = pd.DataFrame(agg_data)
    # Return the Dataframe
    return df_tran

**Aggregate User Data**

In [5]:
# Create the function to extract Aggregate User data, this will accept the path to the aggregate user

def extract_agg_user_data(path):
    #Create a dictionary which contains an empty list to store each items that will be retrieved
    agg_user_data = {
        'State':[], 'Year':[], 'Quarter':[],
        'Registered_user':[], 'App_Open':[], 'Phone_Brand':[], 'Registered_user_by_Phone_Brand':[]
    }
    # loop over each state in the states subfolder
    for state in os.listdir(path):
        # Add each state to the path
        state_path = f"{path}/{state}"
        # loop over each year in the state path derived above
        for year in os.listdir(state_path):
            # add each year to the path
            year_path = f"{state_path}/{year}"
            # loop over the json files
            for json_file in os.listdir(year_path):
                # add each file to the path
                json_path = f"{year_path}/{json_file}"
                # Retreive what quater we're currently extracting, by separating the .json handle
                json_f = json_file.split('.json')[0]
                # convert to a dataframe for easier extraction
                df = pd.read_json(json_path)
                # Chose exactly where we want to extract the data on the dataframe
                data_agg_to_extract = df['data']['aggregated']
                data_to_extract = df['data']['usersByDevice']
                # check if data_to_extract isn't empty, so as to prevent looping through an empty list
                if data_to_extract is not None:
                    # if it's not empty, then loop over it
                    for data in data_to_extract:
                        # Check if it's not empty
                        if data is not None:
                            # Extract the registered users and append it to the appropriate list in the dictionary
                            agg_user_data['Registered_user'].append(data_agg_to_extract['registeredUsers'])
                            # Extract the No of App oepn and append it to the appropriate list in the dictionary
                            agg_user_data['App_Open'].append(data_agg_to_extract['appOpens'])
                            # Extract the Phone Brand and append it to the appropriate list in the dictionary
                            agg_user_data['Phone_Brand'].append(data['brand'])
                            # Extract the Registered users by phone brand and append it to the appropriate list in the dictionary
                            agg_user_data['Registered_user_by_Phone_Brand'].append(data['count'])
                            # Here we retrieve the current state we're extracting from remove uneccesary symbols, clean and Capitalize then append it to the appropriate list in the dictionary
                            agg_user_data['State'].append(state.title().replace('-', ' ').replace('&', 'and'))
                            # Here we retrieve the year and append it to the appropriate list in the dictionary
                            agg_user_data['Year'].append(year)
                            # Reterieve the Quater we're curently extracting from and append it to the appropriate list in the dictionary
                            agg_user_data['Quarter'].append(json_f)
    # We convert the Dictionary into a Dataframe
    df_user = pd.DataFrame(agg_user_data)
    # Return the Dataframe
    return df_user

**Map Transaction Data**

In [6]:
# Create the function to extract Map Transaction data, this will accept the path to the Map Transaction

def extract_map_transact_data(path):
    #Create a dictionary which contains an empty list to store each items that will be retrieved
    map_trans_data = {
        'State':[], 'Year':[], 'Quarter':[],
        'District':[], 'Total_Transaction':[], 'Transaction_Value':[]
    }
    # loop over each state in the states subfolder
    for state in os.listdir(path):
        # Add each state to the path
        state_path = f"{path}/{state}"
        # loop over each year in the state path derived above
        for year in os.listdir(state_path):
            # add each year to the path
            year_path = f"{state_path}/{year}"
            # loop over the json files
            for json_file in os.listdir(year_path):
                # add each file to the path
                json_path = f"{year_path}/{json_file}"
                # Retreive what quater we're currently extracting, by separating the .json handle
                json_f = json_file.split('.json')[0]
                # convert to a dataframe for easier extraction
                df = pd.read_json(json_path)
                # Loop through the data to extract what we want
                for data in df['data']['hoverDataList']:
                    # Extract the District and append it to the appropriate list in the dictionary
                    map_trans_data['District'].append(data['name'].title().replace(' And', ' and').replace('andaman', 'Andaman'))
                    # Extract the Total Transaction and append it to the appropriate list in the dictionary
                    map_trans_data['Total_Transaction'].append(data['metric'][0]['count'])
                    # Extract the Transaction Value and append it to the appropriate list in the dictionary
                    map_trans_data['Transaction_Value'].append(data['metric'][0]['amount'])
                    # Here we retrieve the current state we're extracting from remove uneccesary symbols, clean and Capitalize then append it to the appropriate list in the dictionary
                    map_trans_data['State'].append(state.title().replace('-', ' ').replace('&', 'and'))
                    # Here we retrieve the year and append it to the appropriate list in the dictionary
                    map_trans_data['Year'].append(year)
                    # Reterieve the Quater we're curently extracting from and append it to the appropriate list in the dictionary
                    map_trans_data['Quarter'].append(json_f)
    # We convert the Dictionary into a Dataframe
    df_map_trans = pd.DataFrame(map_trans_data)
    # Return the Dataframe
    return df_map_trans

**Map User Data**

In [7]:
# Create the function to extract Map User data, this will accept the path to the Map User

def extract_map_user_data(path):
    #Create a dictionary which contains an empty list to store each items that will be retrieved
    map_user_data = {
        'State':[], 'Year':[], 'Quarter':[],
        'District':[], 'Total_Registered_User':[], 'Total_App_open':[]
    }
    # loop over each state in the states subfolder
    for state in os.listdir(path):
        # Add each state to the path
        state_path = f"{path}/{state}"
        # loop over each year in the state path derived above
        for year in os.listdir(state_path):
            # add each year to the path
            year_path = f"{state_path}/{year}"
            # loop over the json files
            for json_file in os.listdir(year_path):
                # add each file to the path
                json_path = f"{year_path}/{json_file}"
                # Retreive what quater we're currently extracting, by separating the .json handle
                json_f = json_file.split('.json')[0]
                # convert to a dataframe for easier extraction
                df = pd.read_json(json_path)
                # Loop through the data to extract what we want
                for district, userdata in df['data']['hoverData'].items():
                    # Extract the District and append it to the appropriate list in the dictionary
                    map_user_data['District'].append(district.title().replace(' And', ' and').replace('andaman', 'Andaman'))
                    # Extract the Total registered user and append it to the appropriate list in the dictionary
                    map_user_data['Total_Registered_User'].append(userdata['registeredUsers'])
                    # Extract the Total App opened and append it to the appropriate list in the dictionary
                    map_user_data['Total_App_open'].append(userdata['registeredUsers'])
                    # Here we retrieve the current state we're extracting from remove uneccesary symbols, clean and Capitalize then append it to the appropriate list in the dictionary
                    map_user_data['State'].append(state.title().replace('-', ' ').replace('&', 'and'))
                    # Here we retrieve the year and append it to the appropriate list in the dictionary
                    map_user_data['Year'].append(year)
                    # Reterieve the Quater we're curently extracting from and append it to the appropriate list in the dictionary
                    map_user_data['Quarter'].append(json_f)
    # We convert the Dictionary into a Dataframe
    df_map_user = pd.DataFrame(map_user_data)            
    # Return the Dataframe
    return df_map_user

**Top Transaction**

For Top data, we would need to split our extraction functions, one for district and the other for pincodes because in the dataset they do not have equal amount of data i.e they do not have the same shape(Row)

In [8]:
# Create the function to extract Top Transaction District data, this will accept the path to the Top Transaction

def extract_top_transact_district_data(path):
    #Create a dictionary which contains an empty list to store each items that will be retrieved
    top_transact_district = {
        'State':[], 'Year':[], 'Quarter':[],
        'District':[], 'Total_Transaction': [], 'Transaction_Value': []
    }
    # loop over each state in the states subfolder
    for state in os.listdir(path):
        # Add each state to the path
        state_path = f"{path}/{state}"
        # loop over each year in the state path derived above
        for year in os.listdir(state_path):
            # add each year to the path
            year_path = f"{state_path}/{year}"
            # loop over the json files
            for json_file in os.listdir(year_path):
                # add each file to the path
                json_path = f"{year_path}/{json_file}"
                # Retreive what quater we're currently extracting, by separating the .json handle
                json_f = json_file.split('.json')[0]
                # convert to a dataframe for easier extraction
                df = pd.read_json(json_path)
                # Loop through the data to extract what we want
                for data in df['data']['districts']:
                    # Extract the District and append it to the appropriate list in the dictionary
                    top_transact_district['District'].append(data['entityName'].title().replace(' And', ' and').replace('andaman', 'Andaman'))
                    # Extract the Total Transaction and append it to the appropriate list in the dictionary
                    top_transact_district['Total_Transaction'].append(data['metric']['count'])
                    # Extract the Transaction Value and append it to the appropriate list in the dictionary
                    top_transact_district['Transaction_Value'].append(data['metric']['amount'])
                    # Here we retrieve the current state we're extracting from remove uneccesary symbols, clean and Capitalize then append it to the appropriate list in the dictionary
                    top_transact_district['State'].append(state.title().replace('-', ' ').replace('&', 'and'))
                    # Here we retrieve the year and append it to the appropriate list in the dictionary
                    top_transact_district['Year'].append(year)
                    # Reterieve the Quater we're curently extracting from and append it to the appropriate list in the dictionary
                    top_transact_district['Quarter'].append(json_f)
    # We convert the Dictionary into a Dataframe
    df_top_district = pd.DataFrame(top_transact_district)
    # Return the Dataframe
    return df_top_district

In [9]:
# Create the function to extract Top Transaction Pincode data, this will accept the path to the Top Transaction

def extract_top_transact_pincode_data(path):
    #Create a dictionary which contains an empty list to store each items that will be retrieved
    top_transact_pincode = {
        'State':[], 'Year':[], 'Quarter':[],
        'Pincode':[], 'Total_Transaction': [], 'Transaction_Value': []
    }
    # loop over each state in the states subfolder
    for state in os.listdir(path):
        # Add each state to the path
        state_path = f"{path}/{state}"
        # loop over each year in the state path derived above
        for year in os.listdir(state_path):
            # add each year to the path
            year_path = f"{state_path}/{year}"
            # loop over the json files
            for json_file in os.listdir(year_path):
                # add each file to the path
                json_path = f"{year_path}/{json_file}"
                # Retreive what quater we're currently extracting, by separating the .json handle
                json_f = json_file.split('.json')[0]
                # convert to a dataframe for easier extraction
                df = pd.read_json(json_path)
                # Loop through the data to extract what we want
                for data in df['data']['pincodes']:
                    # Extract the Pincode and append it to the appropriate list in the dictionary
                    # But check if it's not empty
                    if data['entityName'] is not None:
                        # if it's not empty, we convert it to a string and append it to the appropriate list in the dictionary
                        top_transact_pincode['Pincode'].append(data['entityName'].title())
                    else:
                        # if it's empty then we attach the empty string 
                        top_transact_pincode['Pincode'].append(data['entityName'])
                    # Extract the Total Transaction and append it to the appropriate list in the dictionary
                    top_transact_pincode['Total_Transaction'].append(data['metric']['count'])
                    # Extract the Transaction Value and append it to the appropriate list in the dictionary
                    top_transact_pincode['Transaction_Value'].append(data['metric']['amount'])
                    # Here we retrieve the current state we're extracting from remove uneccesary symbols, clean and Capitalize then append it to the appropriate list in the dictionary
                    top_transact_pincode['State'].append(state.title().replace('-', ' ').replace('&', 'and'))
                    # Here we retrieve the year and append it to the appropriate list in the dictionary
                    top_transact_pincode['Year'].append(year)
                    # Reterieve the Quater we're curently extracting from and append it to the appropriate list in the dictionary
                    top_transact_pincode['Quarter'].append(json_f)
    # We convert the Dictionary into a Dataframe
    df_top_pincode = pd.DataFrame(top_transact_pincode)
    # Return the Dataframe
    return df_top_pincode

**Top User Data**

In [10]:
# Create the function to extract Top User District data, this will accept the path to the Top User

def extract_top_user_district_data(path):
    #Create a dictionary which contains an empty list to store each items that will be retrieved
    top_user_district = {
        'State':[], 'Year':[], 'Quarter':[],
        'District':[], 'Registered_Users': []
    }
    # loop over each state in the states subfolder
    for state in os.listdir(path):
        # Add each state to the path
        state_path = f"{path}/{state}"
        # loop over each year in the state path derived above
        for year in os.listdir(state_path):
            # add each year to the path
            year_path = f"{state_path}/{year}"
            # loop over the json files
            for json_file in os.listdir(year_path):
                # add each file to the path
                json_path = f"{year_path}/{json_file}"
                # Retreive what quater we're currently extracting, by separating the .json handle
                json_f = json_file.split('.json')[0]
                # convert to a dataframe for easier extraction
                df = pd.read_json(json_path)
                # Loop through the data to extract what we want
                for data in df['data']['districts']:
                    # Extract the District and append it to the appropriate list in the dictionary
                    top_user_district['District'].append(data['name'].title().replace(' And', ' and').replace('andaman', 'Andaman'))
                    # Extract the Total registered user and append it to the appropriate list in the dictionary
                    top_user_district['Registered_Users'].append(data['registeredUsers'])
                    # Here we retrieve the current state we're extracting from remove uneccesary symbols, clean and Capitalize then append it to the appropriate list in the dictionary
                    top_user_district['State'].append(state.title().replace('-', ' ').replace('&', 'and'))
                    # Here we retrieve the year and append it to the appropriate list in the dictionary
                    top_user_district['Year'].append(year)
                    # Reterieve the Quater we're curently extracting from and append it to the appropriate list in the dictionary
                    top_user_district['Quarter'].append(json_f)
    # We convert the Dictionary into a Dataframe
    df_top_user_district = pd.DataFrame(top_user_district)
    # Return the Dataframe
    return df_top_user_district

In [11]:
# Create the function to extract Top User District data, this will accept the path to the Top User

def extract_top_user_pincode_data(path):
    #Create a dictionary which contains an empty list to store each items that will be retrieved
    top_user_pincode = {
        'State':[], 'Year':[], 'Quarter':[],
        'pincode':[], 'Registered_Users': []
    }
    # loop over each state in the states subfolder
    for state in os.listdir(path):
        # Add each state to the path
        state_path = f"{path}/{state}"
        # loop over each year in the state path derived above
        for year in os.listdir(state_path):
            # add each year to the path
            year_path = f"{state_path}/{year}"
            # loop over the json files
            for json_file in os.listdir(year_path):
                # add each file to the path
                json_path = f"{year_path}/{json_file}"
                # Retreive what quater we're currently extracting, by separating the .json handle
                json_f = json_file.split('.json')[0]
                # convert to a dataframe for easier extraction
                df = pd.read_json(json_path)
                # Loop through the data to extract what we want
                for data in df['data']['pincodes']:
                    # Extract the Pincode and append it to the appropriate list in the dictionary
                    top_user_pincode['pincode'].append(data['name'].title())
                    # Extract the Total registered user and append it to the appropriate list in the dictionary
                    top_user_pincode['Registered_Users'].append(data['registeredUsers'])
                    # Here we retrieve the current state we're extracting from remove uneccesary symbols, clean and Capitalize then append it to the appropriate list in the dictionary
                    top_user_pincode['State'].append(state.title().replace('-', ' ').replace('&', 'and'))
                    # Here we retrieve the year and append it to the appropriate list in the dictionary
                    top_user_pincode['Year'].append(year)
                    # Reterieve the Quater we're curently extracting from and append it to the appropriate list in the dictionary
                    top_user_pincode['Quarter'].append(json_f)
    # We convert the Dictionary into a Dataframe
    df_top_user_pincode = pd.DataFrame(top_user_pincode)
    # Return the Dataframe
    return df_top_user_pincode

***Extract The Dataframes***

In [12]:
# Load and Extract the dataframe

# Now you call all functions to extract the data and convert to a dataframe
agg_trans_df = extract_agg_transact_data(state_paths[0])
agg_user_df = extract_agg_user_data(state_paths[1])
map_trans_df = extract_map_transact_data(state_paths[2])
map_user_df = extract_map_user_data(state_paths[3])
top_trans_dist_df = extract_top_transact_district_data(state_paths[4])
top_trans_pin_df = extract_top_transact_pincode_data(state_paths[4])
top_user_dist_df = extract_top_user_district_data(state_paths[5])
top_user_pin_df = extract_top_user_pincode_data(state_paths[5])

In [13]:
# Put all Dataframes names in a list so we can easily call back

# To the the name of Df(dataframe), we get all items using the globals function, then we filte by checking if it's the instance of a Dataframe and is also ending with _df
df_list = [var_name for var_name, var_value in globals().items() if isinstance(var_value, pd.DataFrame) and var_name.endswith('_df')]
# Here's the list
df_list

['agg_trans_df',
 'agg_user_df',
 'map_trans_df',
 'map_user_df',
 'top_trans_dist_df',
 'top_trans_pin_df',
 'top_user_dist_df',
 'top_user_pin_df']

In [14]:
# Now it was noticed that most district in delhi didn't end with delhi i.e North delhi is North, and it's a bit confusing so we need to fix it. 
# To fix it, we will create a function that will loop through the data and go through the state of delhi and add suffix delhi to the district xcept Shahdara
# Create  function that accepts the dataframe 
def add_delhi(df):
    # First we check if State and Distircts are in the columns of the data frame, because thoseare the required columns for thi function to work
    if 'State' in df.columns and 'District' in df.columns:
        # First we strip the district off the district column, the replace it with nothing, convert it back to a string and set as the new distirct
        df['District'] = df['District'].str.replace('District', '').str.strip()
        # Now we select only data where the state is delhi
        delhi_df = df[df['State'] == 'Delhi']
        # Now we create a list of the unique District but we're exempting Shahdara
        districts_to_suffix = [d for d in delhi_df['District'].unique() if d != 'Shahdara']
        # Now we check for the distirct then change it with the district ending with delhi
        df.loc[(df['State'] == 'Delhi') & (df['District'].isin(districts_to_suffix)), 'District'] = df.loc[(df['State'] == 'Delhi') & (df['District'].isin(districts_to_suffix)), 'District'].apply(lambda x: x + ' Delhi' if 'Delhi' not in x else x)
    # Now return the new data frame
    return df
# Loop over the Dataframe list
for name in df_list:
    # Convert to a Dataframe 
    df = globals()[name]
    # Now call add delhi function
    add_delhi(df)

Now we need to add a geo location data, so i sourced out the data on india on the internet. And we will merge it with our data:

In [15]:
# After converting the data into a CSV file, the we convert it into a dataframe 
Longitude_and_latitude_df = pd.read_csv('csv_files/dist_lat_long.csv')

# Loop over the Dataframe list
for name in df_list:
    # Convert to a Dataframe 
    df = globals()[name]
    # First we check if Districts is in the data frame
    if 'District' in df.columns:
        # Then we merge the current Dataframe with the longitud dataframe on the state and district column
        df = pd.merge(df, Longitude_and_latitude_df, on=['State', 'District'])
        # Assign the dataframe to the to the global dataframe 
        globals()[name] = df

In [16]:
# Now we need to signify the region using the states, to do that we will need to creat a function to add the region

# Create  function that accepts the dataframe
def add_region(df):
    # Now we have create a dictionary with the region as a key an the list of states in it
    state_groups = {
        'Northern Region': ['Jammu and Kashmir', 'Himachal Pradesh', 'Punjab', 'Chandigarh', 'Uttarakhand', 'Ladakh', 'Delhi', 'Haryana'],
        'Central Region': ['Uttar Pradesh', 'Madhya Pradesh', 'Chhattisgarh'],
        'Western Region': ['Rajasthan', 'Gujarat', 'Dadra and Nagar Haveli and Daman and Diu', 'Maharashtra'],
        'Eastern Region': ['Bihar', 'Jharkhand', 'Odisha', 'West Bengal', 'Sikkim'],
        'Southern Region': ['Andhra Pradesh', 'Telangana', 'Karnataka', 'Kerala', 'Tamil Nadu', 'Puducherry', 'Goa', 'Lakshadweep', 'Andaman and Nicobar Islands'],
        'North-Eastern Region': ['Assam', 'Meghalaya', 'Manipur', 'Nagaland', 'Tripura', 'Arunachal Pradesh', 'Mizoram']
    }

    # Loop throught the dictionary
    for region, states in state_groups.items():
        # Now we check if the state in the region then we add that region on a new column, Region
        df.loc[df['State'].isin(states), 'Region'] = region
    # Now you return the New Dataframe
    return df


In [17]:
# Loop over the Dataframe list
for name in df_list:
    # Convert to a Dataframe 
    df = globals()[name]
    # Now add the regio using the function create earlier
    add_region(df)
    # Assign the dataframe to the to the global dataframe 
    globals()[name] = df

In [18]:
# Now we have gotten the datafrmes ready, then we check for null alues ambiguities etc...

# Loop over the Dataframe list
for name in df_list:
    # Convert to a Dataframe
    df = globals()[name]
    # Get the dataframe name
    print(f'{name}:')
    # add a seperator
    print(25 * '_')
    # Now get the Null Values
    print(f'{df.isnull().sum()}')
    # Now check for duplicated values
    print(f'Duplicated Rows: {df.duplicated().sum()}')
    # Also print out the shapes, row and column
    print(f'Dataframe Shape: {df.shape}')
    # Add a separator
    print(25 * '*')

agg_trans_df:
_________________________
State                0
Year                 0
Quarter              0
Payment_Type         0
Total_Transaction    0
Transaction_Value    0
Region               0
dtype: int64
Duplicated Rows: 0
Dataframe Shape: (3594, 7)
*************************
agg_user_df:
_________________________
State                             0
Year                              0
Quarter                           0
Registered_user                   0
App_Open                          0
Phone_Brand                       0
Registered_user_by_Phone_Brand    0
Region                            0
dtype: int64
Duplicated Rows: 0
Dataframe Shape: (6732, 8)
*************************
map_trans_df:
_________________________
State                0
Year                 0
Quarter              0
District             0
Total_Transaction    0
Transaction_Value    0
Latitude             0
Longitude            0
Region               0
dtype: int64
Duplicated Rows: 0
Dataframe Shape: (14636

In [19]:
# I noticed that top_trans_pin_df has pincode with 2 missing data, so i nee to remove it 

# Firstly, since its insignificant then we remove those rows 
top_trans_pin_df.dropna(subset=['Pincode'], inplace=True)

# Then we check again
top_trans_pin_df.isnull().sum()


State                0
Year                 0
Quarter              0
Pincode              0
Total_Transaction    0
Transaction_Value    0
Region               0
dtype: int64

In [20]:
# Now lets check for the Dataframe information
print('Dataframe Info: \n')
# Loop over the Dataframe list
for name in df_list:
    # Convert to a Dataframe
    df = globals()[name]
    # print the name of the dataframe
    print(f'{name}')
    # Print out the information for the Data frame
    df.info()
    # add a separator
    print(f'\n', 45 * '*', '\n')

Dataframe Info: 

agg_trans_df
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3594 entries, 0 to 3593
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   State              3594 non-null   object 
 1   Year               3594 non-null   object 
 2   Quarter            3594 non-null   object 
 3   Payment_Type       3594 non-null   object 
 4   Total_Transaction  3594 non-null   int64  
 5   Transaction_Value  3594 non-null   float64
 6   Region             3594 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 196.7+ KB

 ********************************************* 

agg_user_df
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6732 entries, 0 to 6731
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   State                           6732 non-null   object
 1   Year          

In [21]:
# Now we see that year appars as an object instead of beinf and int, we solve that
# Loop over the Dataframe list
for name in df_list:
    # Convert to a Dataframe
    df = globals()[name]
    # Convert year to an int type
    df['Year'] = df['Year'].astype('int')


In [22]:
# Now lets check for the Dataframe information
print('Dataframe Info: \n')
# Loop over the Dataframe list
for name in df_list:
    # Convert to a Dataframe
    df = globals()[name]
    # print the name of the dataframe
    print(f'{name}')
    # Print out the information for the Data frame
    df.info()
    # add a separator
    print(f'\n', 45 * '*', '\n')

Dataframe Info: 

agg_trans_df
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3594 entries, 0 to 3593
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   State              3594 non-null   object 
 1   Year               3594 non-null   int32  
 2   Quarter            3594 non-null   object 
 3   Payment_Type       3594 non-null   object 
 4   Total_Transaction  3594 non-null   int64  
 5   Transaction_Value  3594 non-null   float64
 6   Region             3594 non-null   object 
dtypes: float64(1), int32(1), int64(1), object(4)
memory usage: 182.6+ KB

 ********************************************* 

agg_user_df
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6732 entries, 0 to 6731
Data columns (total 8 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   State                           6732 non-null   object
 1   Year

In [23]:
# Now we need to check for outliers on all data frames, so we need a function that will count the outliers.

# Create  function that accepts the dataframe
def count_outliers(df):
    # Create an empty dictionary to hold each column and the number of outliers they have
    outliers = {}
    # We loop throgh the dataframe's columns which has numbers in it, meaning int, float etc.
    for col in df.select_dtypes(include=[np.number]).columns:
        # Now we check if the column is either a Total Transaction or a Transaction Value
        if col in ['Total_Transaction', 'Transaction_Value']:
            # If it's either of the two columns then we proceed
            # Now we calculate the first quantile
            q1 = df[col].quantile(0.25)
            # Now we calculate the third quantile
            q3 = df[col].quantile(0.75)
            # Then we proceed to calculate the interquatile range
            iqr = q3 - q1
            # Now we calculate the upper and lower bound
            # Calculate the Upper bound
            upper_bound = q3 + (1.5 * iqr)
            # Calculate the Lower bound
            lower_bound = q1 - (1.5 * iqr)
            # Now we attach to the dictionary, the lenght of the data greater than the upper bound and lesser than the lower bound
            outliers[col] = len(df[(df[col] > upper_bound) | (df[col] < lower_bound)])
        # If it's neither of the two columns then we proceed
        else:
            # Continue
            continue
    # Now we return the dictionary
    return outliers

In [24]:
# Loop over the Dataframe list
for name in df_list:
    # Convert to a Dataframe
    df = globals()[name]
    # Now we run the function to get the count of our outliers and assign to a vairable
    outliers = count_outliers(df)
    # Now we check if th outlier isn't equal to zero
    if len(outliers) == 0:
        # If it is we pass
        pass
    else:
        # If it's not, then we print out the name of the data frame and then the outliers
        print(name, ":\n\n", outliers, "\n")
        # add a separator
        print("\n", 55 * "_", "\n")

agg_trans_df :

 {'Total_Transaction': 652, 'Transaction_Value': 660} 


 _______________________________________________________ 

map_trans_df :

 {'Total_Transaction': 1811, 'Transaction_Value': 1771} 


 _______________________________________________________ 

top_trans_dist_df :

 {'Total_Transaction': 734, 'Transaction_Value': 743} 


 _______________________________________________________ 

top_trans_pin_df :

 {'Total_Transaction': 999, 'Transaction_Value': 995} 


 _______________________________________________________ 



After checking for the outliers then we noticed that 4 DF's have outliers. Then we proceed to check for unique values 

In [25]:
# Now we need to check for unique variables, so we will write a function to print the unique values

# Create  function that accepts the dataframe, and excluded column
def unique_values(df, exclude_cols=[]):
    # Loop throught the dataframe's column
    for col in df.columns:
        # Check if the column is in the excluded colun list
        if col in exclude_cols:
            # If it is, then we we pass
            continue
        # If not then we proceed to extract the unique values
        else:
            #First, we get the unique values in the number of unique values in the column
            unique_vals = df[col].nunique()
            # Then we print the column followed by the number of unique values
            print(f"{col}: {unique_vals} unique values")
            # Now, check If the uniuqe value is lesser than 10
            if unique_vals < 10:
                # If it is then print the unique values
                print(df[col].unique())

In [26]:
# Now lets check the unique values of all data frames

# Let's print the tittle
print('Unique Values in Accross the Data Frame: \n')

# Loop over the Dataframe list
for name in df_list:
    # Convert to a Dataframe
    df = globals()[name]
    # Print the name
    print(f'{name}: \n')
    # Call the unique values function
    unique_values(df, exclude_cols=['State', 'Year', 'Quarter'])
    # add a separator
    print("\n", 55 * "*", "\n")

Unique Values in Accross the Data Frame: 

agg_trans_df: 

Payment_Type: 5 unique values
['Recharge & bill payments' 'Peer-to-peer payments' 'Merchant payments'
 'Financial Services' 'Others']
Total_Transaction: 3548 unique values
Transaction_Value: 3594 unique values
Region: 6 unique values
['Southern Region' 'North-Eastern Region' 'Eastern Region'
 'Northern Region' 'Central Region' 'Western Region']

 ******************************************************* 

agg_user_df: 

Registered_user: 612 unique values
App_Open: 433 unique values
Phone_Brand: 20 unique values
Registered_user_by_Phone_Brand: 6501 unique values
Region: 6 unique values
['Southern Region' 'North-Eastern Region' 'Eastern Region'
 'Northern Region' 'Central Region' 'Western Region']

 ******************************************************* 

map_trans_df: 

District: 727 unique values
Total_Transaction: 14566 unique values
Transaction_Value: 14636 unique values
Latitude: 532 unique values
Longitude: 540 unique values

In [27]:
# Now let's save all data frames, converting it into a csv file

# Create  function that accepts the dataframe list
def save_csv(df_list):
    # Select the subfolders where you want the files to be 
    subfolder = "C:/Users/a/OneDrive/Desktop/Github_projects/DS_Python_projects/Dashboard Project/csv_files"
    # Now if the subfolder path dosen't exist
    if not os.path.exists(subfolder):
        # If it's not then create it
        os.makedirs(subfolder) 
        # loop over the list 
    for df_name in df_list:
        # Convert to a Dataframe
        df = globals()[df_name]
        #Create a file path, by joining the subfolder path with the derived file name ending with CSV
        file_path = os.path.join(subfolder, df_name.replace('_df', '') + '.csv')
        # Convert to  CSV file using the path
        df.to_csv(file_path, index=False)
    # Print a succesful message 
    print('Successful')

In [28]:
# Call the function 
save_csv(df_list)

Successful


Now we have been able to Extract, clean and prepae our data. Now we need to push it into our database for easy extraction and accesibility

In [29]:
# First we need to connect to our database, and to that we will need our loggin details. But because those details are sensitive we will input them using get pass
# First we get credentials, we already know the username we now need to get the username
# Print prompt
print('Please Enter your Password: ')

# Now to secure the password we use get pass to encode it
my_password = getpass("Password: ")

#Now we have the credentials, let's connect to the database using the Mysql collector
conn = mysql.connector.connect(
  # Get the host
  host = "localhost",
  # Get the Username
  user = "root",
  # Get the password
  password = f"{my_password}"
)

# Now we connect to our cursor
cursor = conn.cursor()


Please Enter your Password: 
Password: ········


In [30]:
# Now let's drop th database if it exist on the Database
cursor.execute("DROP DATABASE IF EXISTS phone_pulse")
# Now We create the database
cursor.execute("CREATE DATABASE phone_pulse")
# Then we need to select the newly created database 
cursor.execute("USE phone_pulse")

In [31]:
# Now we create Tables for all dataframes on our database
cursor.execute('''CREATE TABLE agg_trans (
                    State VARCHAR(250),
                    Year YEAR,
                    Quarter INTEGER,
                    Payment_Type VARCHAR(250),
                    Total_Transaction INTEGER,
                    Transaction_Value FLOAT,
                    Region VARCHAR(250),
                    PRIMARY KEY (State(250), Year, Quarter, Payment_type(250), Region(250))
                )''')
cursor.execute('''CREATE TABLE agg_user (
                    State VARCHAR(250), 
                    Year YEAR, 
                    Quarter INTEGER, 
                    Registered_user INTEGER, 
                    App_open BIGINT, 
                    Phone_Brand VARCHAR(250), 
                    Registered_user_by_Phone_Brand INTEGER, 
                    Region VARCHAR(250), 
                    PRIMARY KEY (State(250), Year, Quarter, Phone_Brand(250), Region(250))
                )''')
cursor.execute('''CREATE TABLE map_trans (
                    State VARCHAR(250), 
                    Year YEAR, 
                    Quarter INTEGER, 
                    District VARCHAR(250), 
                    Total_Transaction INTEGER, 
                    Transaction_Value FLOAT, 
                    Latitude FLOAT, 
                    Longitude FLOAT, 
                    Region VARCHAR(250),
                    PRIMARY KEY (State(250), Year, Quarter, District(250), Region(250))
                )''')
cursor.execute('''CREATE TABLE map_user (
                    State VARCHAR(250), 
                    Year YEAR, 
                    Quarter INTEGER, 
                    District VARCHAR(250), 
                    Total_Registered_User INTEGER, 
                    Total_App_open INTEGER, 
                    Latitude FLOAT, 
                    Longitude FLOAT, 
                    Region VARCHAR(250), 
                    PRIMARY KEY (State(250), Year, Quarter, District(250), Region(250))
                )''')
cursor.execute('''CREATE TABLE top_trans_dist (
                    State VARCHAR(250), 
                    Year YEAR, 
                    Quarter INTEGER, 
                    District VARCHAR(250), 
                    Total_Transaction INTEGER, 
                    Transaction_Value FLOAT, 
                    Latitude FLOAT, 
                    Longitude FLOAT, 
                    Region VARCHAR(250),
                    PRIMARY KEY (State(250), Year, Quarter, District(250), Region(250))
                )''')
cursor.execute('''CREATE TABLE top_trans_pin (
                    State VARCHAR(250), 
                    Year YEAR, 
                    Quarter INTEGER, 
                    Pincode VARCHAR(250), 
                    Total_Transaction INTEGER, 
                    Transaction_Value FLOAT, 
                    Region VARCHAR(250),
                    PRIMARY KEY (State(250), Year, Quarter, Pincode(250), Region(250))
                )''')
cursor.execute('''CREATE TABLE top_user_dist (
                    State VARCHAR(250), 
                    Year YEAR, 
                    Quarter INTEGER, 
                    District VARCHAR(250), 
                    Registered_Users INTEGER, 
                    Latitude FLOAT, 
                    Longitude FLOAT, 
                    Region VARCHAR(250),
                    PRIMARY KEY (State(250), Year, Quarter, District(250), Region(250))
                )''')
cursor.execute('''CREATE TABLE top_user_pin (
                    State VARCHAR(250), 
                    Year YEAR, 
                    Quarter INTEGER, 
                    Pincode VARCHAR(250), 
                    Registered_Users INTEGER, 
                    Region VARCHAR(250),
                    PRIMARY KEY (State(250), Year, Quarter, Pincode(250), Region(250))
                )''')

In [32]:
# Now we need to verify if the tables have been uploaded to the database

# Execute query to show tables in the database
cursor.execute("SHOW TABLES;")
# Fetch all the data from the query  
databases = cursor.fetchall()
# loop through the data
for db in databases:
    # Print the database
    print(db[0])

agg_trans
agg_user
map_trans
map_user
top_trans_dist
top_trans_pin
top_user_dist
top_user_pin


Now we have created the database, created the Tables next thing we need to do is push the data on the dataframe to respective tables on our database

In [33]:
# First we need to get our DFS as a dictionary and also put our Table columns also in a dictionary
dfs = {
    'agg_trans':agg_trans_df,
    'agg_user':agg_user_df,
    'map_trans':map_trans_df,
    'map_user':map_user_df,
    'top_trans_dist':top_trans_dist_df,
    'top_trans_pin':top_trans_pin_df,
    'top_user_dist':top_user_dist_df,
    'top_user_pin':top_user_pin_df
}

table_columns = {
    'agg_trans': list(agg_trans_df.columns),
    'agg_user': list(agg_user_df.columns),
    'map_trans': list(map_trans_df.columns),
    'map_user': list(map_user_df.columns),
    'top_trans_dist': list(top_trans_dist_df.columns),
    'top_trans_pin': list(top_trans_pin_df.columns),
    'top_user_dist': list(top_user_dist_df.columns),
    'top_user_pin': list(top_user_pin_df.columns)
}

In [34]:
# Now we will write a function to push the data to the database

# Create a function that accepts the connection, cursor, dataframe dictonary, and table columns dictionary
def push_data_into_mysql(conn, cursor, dfs, table_columns):
    # We loop through the keys in the df dictionary, hereby looping through the list names
    for table_name in dfs.keys():
        # Now we extract the dataframe
        df = dfs[table_name]
        # We extract the columns
        columns = table_columns[table_name]
        # Now we create placeholders for Insert into query
        placeholders = ', '.join(['%s'] * len(columns))
        # Create a body or emplate query where the dat will be inserted
        query = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({placeholders})"
        # Now we loop over the rows of the dataframe and extract just the data
        for _, row in df.iterrows():
            # Now we also loop through the columns then insert them in the row to extract the exact data we need, then save all in a tuple to avoid mixing it up, since they cant modified
            data = tuple(row[column] for column in columns)
            # Now we execute the query, also adding the respective data
            cursor.execute(query, data)
        # Next we commit to the connection
        conn.commit()
    # Print Success message, to show the function ran sucessful
    print("Data successfully pushed into MySQL tables")

In [35]:
# Call the function with all it's dependencies 
push_data_into_mysql(conn, cursor, dfs, table_columns)

Data successfully pushed into MySQL tables


In [36]:
# Now we need to verify if all data was uploaded sucessfully by cross checking with the dataframes and the table

# Execute the query to show table in the current Database
cursor.execute("SHOW TABLES")
# Fetch all tables
tables = cursor.fetchall()

# Loop through the Tables
for table in tables:
    # Extract the table
    table_name = table[0]
    # Execute the query to count the number of rows of the table
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    # Store the result in a variable
    row_count = cursor.fetchone()[0]
    # execute the query to count the columns in the table
    cursor.execute(f"SELECT COUNT(*) FROM information_schema.columns WHERE table_name='{table_name}'")
    # store the count in a variable 
    column_count = cursor.fetchone()[0]


    # From the Dataframe select the exact dataframe using the table
    df = dfs[table_name]
    # Now we check if the Dataframe shape is equal to the Table row and column count
    if df.shape == (row_count, column_count):
        # If True print this
        print(f"{table_name} table has {row_count} rows and {column_count} columns and it's shape matches with the DataFrame.")
    else:
        # If False print this
        print(f"{table_name} table has {row_count} rows and {column_count} columns but it's shape does not match with the DataFrame.")

# After verification, close connection
# Close cursor
cursor.close()
# Then close the entire connection
conn.close()

agg_trans table has 3594 rows and 7 columns and it's shape matches with the DataFrame.
agg_user table has 6732 rows and 8 columns and it's shape matches with the DataFrame.
map_trans table has 14636 rows and 9 columns and it's shape matches with the DataFrame.
map_user table has 14640 rows and 9 columns and it's shape matches with the DataFrame.
top_trans_dist table has 5920 rows and 9 columns and it's shape matches with the DataFrame.
top_trans_pin table has 7137 rows and 7 columns and it's shape matches with the DataFrame.
top_user_dist table has 5920 rows and 8 columns and it's shape matches with the DataFrame.
top_user_pin table has 7140 rows and 6 columns and it's shape matches with the DataFrame.


Now our data has been uploaded to our dataframes has been uploaded to our Database and the all have the same shape witht the Dataframe